#Founders data

In [1]:
!pip install Faker

In [2]:
import pandas as pd
import random
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn import set_config
from faker import Faker
import numpy as np
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import copy
import pickle

In [3]:
fake=Faker()

In [4]:
num_profiles = 500
job_titles = ["CEO", "Product Manager", "Startup Founder", "Founder", "CTO", "CFO"]
locations = ["San Francisco", "Amsterdam","Utrecht","Berlin", "New York", "Seoul","Madrid","Munich","Barcelona","Warsaw","Helsinki","London", "Los Angeles"]
languages = ["English", "Spanish", "Finnish","German","Arabic","Dutch","Korean"]
business_field=["IT","Big Data","Sales","Logistics", "Web3", "AI", "Finance","Consulting"]
skills = ["Python", "Machine Learning", "MATLAB", "Product Management","Excel", "Power BI", "Team Work", "SCRUM", "SQL", "RevOps"]
education = ["Stanford University","Utrecht University","Caltech", "Yale", "Oxford", "ETH Zürich", "Berkeley", "Ágora" ,"Harvard University", "MIT"]




data = {
    "Founder ID": [id for id in range(num_profiles)],
    "Full Name": [fake.name() for _ in range(num_profiles)],
    "Job Title": [random.choice(job_titles) for _ in range(num_profiles)],
    "Years of Experience": [round(random.uniform(0, 5), 1) for _ in range(num_profiles)],
    "Location": [random.choice(locations) for _ in range(num_profiles)],
    "Business Field": [random.choice(business_field) for _ in range (num_profiles)],
    "Skills": [random.sample(skills, random.randint(1, len(skills))) for _ in range(num_profiles)],
    "Skill Endorsements": [random.randint(0,100) for _ in range(num_profiles)],
    "Languages": [random.sample(languages, random.randint(1, 3)) for _ in range(num_profiles)],
    "Education":[random.choice(education)for _ in range(num_profiles)
    ],

}
test_data = {
    "Founder ID": [num_profiles+id for id in range(num_profiles)],
    "Full Name": [fake.name() for _ in range(num_profiles)],
    "Job Title": [random.choice(job_titles) for _ in range(num_profiles)],
    "Years of Experience": [round(random.uniform(0, 5), 1) for _ in range(num_profiles)],
    "Location": [random.choice(locations) for _ in range(num_profiles)],
    "Business Field": [random.choice(business_field) for _ in range (num_profiles)],
    "Skills": [random.sample(skills, random.randint(1, len(skills))) for _ in range(num_profiles)],
    "Skill Endorsements": [random.randint(0,100) for _ in range(num_profiles)],
    "Languages": [random.sample(languages, random.randint(1, 3)) for _ in range(num_profiles)],
    "Education":[random.choice(education)for _ in range(num_profiles)
    ],
}

df_founders = pd.DataFrame(data)
test_df_founders = pd.DataFrame(test_data)
df_founders=df_founders.set_index('Founder ID')
test_df_founders=test_df_founders.set_index('Founder ID')

In [5]:
df_founders.head()

,Full Name,Job Title,Years of Experience,Location,Business Field,Skills,Skill Endorsements,Languages,Education
Founder ID,,,,,,,,,
0,Claudia Bell,CFO,1.3,New York,Web3,"[MATLAB, SQL, RevOps]",19,[German],ETH Zürich
1,Steven Todd,Product Manager,4.7,Los Angeles,Big Data,"[RevOps, Machine Learning, Python, Power BI, P...",66,[Spanish],Caltech
2,Theresa Chung,CFO,4.3,Munich,IT,[MATLAB],6,"[Finnish, Dutch]",Caltech
3,Gabriel Eaton,CFO,2.9,Berlin,Consulting,[MATLAB],83,"[Spanish, Korean]",Utrecht University
4,William Rodriguez,Founder,4.6,New York,Big Data,"[SCRUM, Excel, Python, Team Work, RevOps, SQL,...",16,[Dutch],Berkeley


Now I need to convert all of the columns with multiple values into binary matrixes

In [6]:
df_founders.columns, test_df_founders.columns

(Index(['Full Name', 'Job Title', 'Years of Experience', 'Location',
        'Business Field', 'Skills', 'Skill Endorsements', 'Languages',
        'Education'],
       dtype='object'),
 Index(['Full Name', 'Job Title', 'Years of Experience', 'Location',
        'Business Field', 'Skills', 'Skill Endorsements', 'Languages',
        'Education'],
       dtype='object'))

Convert the columns with objects into binary matrices (dummies)

In [7]:
df_founders = df_founders.drop(columns='Skills').join(df_founders['Skills'].str.join('|').str.get_dummies('|'))
df_founders = df_founders.drop(columns='Languages').join(df_founders['Languages'].str.join('|').str.get_dummies('|'))

In [8]:
test_df_founders = test_df_founders.drop(columns='Skills').join(test_df_founders['Skills'].str.join('|').str.get_dummies('|'))
test_df_founders = test_df_founders.drop(columns='Languages').join(test_df_founders['Languages'].str.join('|').str.get_dummies('|'))

In [9]:
test_df_founders.tail()

,Full Name,Job Title,Years of Experience,Location,Business Field,Skill Endorsements,Education,Excel,MATLAB,Machine Learning,...,SCRUM,SQL,Team Work,Arabic,Dutch,English,Finnish,German,Korean,Spanish
Founder ID,,,,,,,,,,,,,,,,,,,,,
995,Christie Hale,Startup Founder,5.0,New York,Consulting,88,Stanford University,1,0,0,...,0,1,0,1,0,0,0,0,0,1
996,Jonathan Murphy,Product Manager,4.1,Barcelona,IT,48,Yale,1,0,1,...,0,0,0,1,0,0,0,0,1,0
997,James Goodwin,Product Manager,2.5,London,AI,20,MIT,1,1,0,...,0,1,0,0,0,1,0,0,0,0
998,Sarah Smith,Startup Founder,4.7,New York,Consulting,84,ETH Zürich,0,0,1,...,0,0,0,1,0,1,0,0,0,0
999,Melissa Gonzalez,CTO,3.1,Helsinki,Finance,5,Stanford University,0,1,1,...,1,0,1,1,0,0,0,1,0,0


Export the CSV for future use

In [10]:
df_founders.to_csv("founders.csv", index=False)
test_df_founders.to_csv("test_founders.csv", index=False)

In [11]:
categorical_cols = df_founders.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df_founders.select_dtypes(include='number').columns.tolist()

In [12]:
founders_categorical_copy=copy.copy(categorical_cols)

In [13]:
len(categorical_cols)+len(numeric_cols)

24

I will run the data preprocessing pipeline on both DataFrames separately, as the founders and cofounders (users and items) and their interactions cannot be repeated, since the internal mapping of the model will calculate the weights prior to the fit.

In [14]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(),
)])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [15]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols),
])

In [16]:
set_config(display="diagram")
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
])
my_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Years of Experience',
                                                   'Skill Endorsements',
                                                   'Excel', 'MATLAB',
                                                   'Machine Learning',
                                                   'Power BI',
                                                   'Product Management',
                                                   'Python', 'RevOps', 'SCRUM',
                                                   'SQL', 'Team Work', 'Arabic',
                                                   'Dutch', 'English',
                                                   'Finnish', 'German',
                                                   'Korean', 'Spanish']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Full Name', 'Job Title',
                                                   'Location', 'Business Field',
                                                   'Education'])]))])

In [17]:
df_founders_preprocessed=my_pipeline.fit_transform(df_founders)
test_df_founders_preprocessed=my_pipeline.fit_transform(test_df_founders)

In [18]:
type(df_founders_preprocessed)

scipy.sparse._csr.csr_matrix

In [19]:
df_founders_preprocessed.shape,test_df_founders_preprocessed.shape

((500, 553), (500, 554))

Here I can confirm that the data has been transformed into a sparse matrix, which we will need to create a similarity matrix for my model's interactions input

In [20]:
type(df_founders_preprocessed)

scipy.sparse._csr.csr_matrix

Save the sparse matrices

In [21]:
sparse.save_npz("founders_sparse_matrix.npz", df_founders_preprocessed)
sparse.save_npz("test_founders_sparse_matrix.npz", test_df_founders_preprocessed)


In [22]:
founders_matrix = sparse.load_npz("founders_sparse_matrix.npz")
test_founders_matrix = sparse.load_npz("test_founders_sparse_matrix.npz")

#Co-founders data

The process for creating the co-founder matrix will be identical but it will have a smaller scope due to the smaller sample of scraped LinkedIn users that fit this profile

In [23]:
num_profiles = 100
job_titles = ["Mentor", "Executive Member", "Chief of Sales", "Entrepeneur", "Chief Executive", "Angel Investor"]
locations = ["Palo Alto", "Amsterdam","Cologne","Boston","London", "Tel Aviv"]
languages = ["English", "Spanish", "Finnish","German","Arabic","Dutch"]
expertise=["Product Development","Business Intelligence","Marketing & Branding","Mentorship","Entrepeneurship","Venture Capital","IT","Regulatory Affairs","Sustainability","Non-profit"]



data = {
    "Co-founder ID": [id for id in range(num_profiles)],
    "Full Name": [fake.name() for _ in range(num_profiles)],
    "Job Title": [random.choice(job_titles) for _ in range(num_profiles)],
    "Field of Expertise":[random.sample(expertise,random.randint(1,5)) for _ in range(num_profiles)],
    "Years of Experience": [round(random.uniform(10, 45), 1) for _ in range(num_profiles)],
    "Languages": [random.sample(languages, random.randint(1, 2)) for _ in range(num_profiles)],
    "Capital in $1K": [random.randrange(0,1000,10) for _ in range(num_profiles)],
    "Location": [random.choice(locations) for _ in range(num_profiles)],
    "Previous Companies":[random.randint(0, 10) for _ in range(num_profiles)],

}
test_data = {
    "Co-founder ID": [num_profiles+id for id in range(num_profiles)],
    "Full Name": [fake.name() for _ in range(num_profiles)],
    "Job Title": [random.choice(job_titles) for _ in range(num_profiles)],
    "Field of Expertise":[random.sample(expertise,random.randint(1,5)) for _ in range(num_profiles)],
    "Years of Experience": [round(random.uniform(10, 45), 1) for _ in range(num_profiles)],
    "Languages": [random.sample(languages, random.randint(1, 2)) for _ in range(num_profiles)],
    "Capital in $1K": [random.randrange(0,1000,10) for _ in range(num_profiles)],
    "Location": [random.choice(locations) for _ in range(num_profiles)],
    "Previous Companies":[random.randint(0, 10) for _ in range(num_profiles)],

}
df_cofounders=pd.DataFrame(data)
test_df_cofounders=pd.DataFrame(test_data)

In [24]:
df_cofounders.columns,test_df_cofounders.columns

(Index(['Co-founder ID', 'Full Name', 'Job Title', 'Field of Expertise',
        'Years of Experience', 'Languages', 'Capital in $1K', 'Location',
        'Previous Companies'],
       dtype='object'),
 Index(['Co-founder ID', 'Full Name', 'Job Title', 'Field of Expertise',
        'Years of Experience', 'Languages', 'Capital in $1K', 'Location',
        'Previous Companies'],
       dtype='object'))

In [25]:
df_cofounders = df_cofounders.drop(columns='Field of Expertise').join(df_cofounders['Field of Expertise'].str.join('|').str.get_dummies('|'))
df_cofounders = df_cofounders.drop(columns='Languages').join(df_cofounders['Languages'].str.join('|').str.get_dummies('|'))

In [26]:
test_df_cofounders = test_df_cofounders.drop(columns='Field of Expertise').join(test_df_cofounders['Field of Expertise'].str.join('|').str.get_dummies('|'))
test_df_cofounders = test_df_cofounders.drop(columns='Languages').join(test_df_cofounders['Languages'].str.join('|').str.get_dummies('|'))

In [27]:
df_cofounders.tail()

,Co-founder ID,Full Name,Job Title,Years of Experience,Capital in $1K,Location,Previous Companies,Business Intelligence,Entrepeneurship,IT,...,Product Development,Regulatory Affairs,Sustainability,Venture Capital,Arabic,Dutch,English,Finnish,German,Spanish
95,95,Yvette Johnson,Angel Investor,30.4,620,Tel Aviv,10,0,0,0,...,1,1,1,0,0,0,0,0,1,0
96,96,Megan Grant,Chief Executive,23.7,10,London,6,0,0,0,...,0,0,1,0,1,0,1,0,0,0
97,97,Brian Simpson,Angel Investor,38.8,790,Boston,3,0,0,0,...,0,0,0,0,0,0,1,0,0,1
98,98,Crystal Miller,Entrepeneur,42.4,20,Boston,5,0,0,0,...,0,1,1,1,0,1,0,1,0,0
99,99,Jennifer Gordon,Angel Investor,33.4,960,Cologne,9,1,0,0,...,0,0,0,0,1,0,0,1,0,0


In [28]:
df_cofounders.to_csv("cofounders.csv", index=False)
test_df_cofounders.to_csv("test_cofounders.csv", index=False)

In [29]:
categorical_cols = df_cofounders.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df_cofounders.select_dtypes(include='number').columns.tolist()

In [30]:
cofounders_categorical_copy=copy.copy(categorical_cols)

In [31]:
len(categorical_cols)+len(numeric_cols)

23

I'm going to use the exact same preprocessing transformers as the data is very similar both in content and shape

In [32]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols),
])

In [33]:
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
])
my_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Co-founder ID',
                                                   'Years of Experience',
                                                   'Capital in $1K',
                                                   'Previous Companies',
                                                   'Business Intelligence',
                                                   'Entrepeneurship', 'IT',
                                                   'Marketing & Branding',
                                                   'Mentorship', 'Non-profit',
                                                   'Product Development',
                                                   'Regulatory Affairs',
                                                   'Sustainability',
                                                   'Venture Capital', 'Arabic',
                                                   'Dutch', 'English',
                                                   'Finnish', 'German',
                                                   'Spanish']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Full Name', 'Job Title',
                                                   'Location'])]))])

In [34]:
df_cofounders_preprocessed=my_pipeline.fit_transform(df_cofounders)
test_df_cofounders_preprocessed=my_pipeline.fit_transform(test_df_cofounders)

In [35]:
df_cofounders_preprocessed.shape,test_df_cofounders_preprocessed.shape

((100, 132), (100, 132))

In [36]:
sparse.save_npz("cofounders_sparse_matrix.npz", df_cofounders_preprocessed)
sparse.save_npz("test_cofounders_sparse_matrix.npz", test_df_cofounders_preprocessed)


In [37]:
cofounders_matrix = sparse.load_npz("cofounders_sparse_matrix.npz")
test_cofounders_matrix = sparse.load_npz("test_cofounders_sparse_matrix.npz")

In [38]:
founders_matrix,cofounders_matrix

(<500x553 sparse matrix of type '<class 'numpy.float64'>'
 	with 12000 stored elements in Compressed Sparse Row format>,
 <100x132 sparse matrix of type '<class 'numpy.float64'>'
 	with 2300 stored elements in Compressed Sparse Row format>)

SINGULAR VALUE DECOMPOSITION FOR OUR SPARSE MATRICES

Similarity matrix

In [39]:
# K(number of latent values) will be the size of the cofounders (n_cofounders+features) dimensity
k = df_cofounders_preprocessed.shape[1]
U, Sigma, VT = svds(founders_matrix, k)


Sigma = np.diag(Sigma)

# Compute the reduced user and item matrices
reduced_founders_matrix = np.dot(U, Sigma)

In [40]:
founder_cofounder_similarity = cosine_similarity(reduced_founders_matrix, cofounders_matrix)

Test similarity matrix

In [41]:
k = test_df_cofounders_preprocessed.shape[1]
U, Sigma, VT = svds(test_founders_matrix, k)


Sigma = np.diag(Sigma)


test_reduced_founders_matrix = np.dot(U, Sigma)

In [42]:
test_founder_cofounder_similarity = cosine_similarity(test_reduced_founders_matrix, test_cofounders_matrix)

In [43]:
founder_cofounder_similarity.shape , test_founder_cofounder_similarity.shape

((500, 100), (500, 100))

#Model

For this challenge I chose the [LightFM](https://making.lyst.com/lightfm/docs/home.html) model, a hybrid recommendation system that combines multiple recommendation algorithms for both implicity and explicit feedback.

In [44]:
!pip install lightfm

In [45]:
from lightfm import LightFM,evaluation,cross_validation
from lightfm.data import Dataset

Once I have imported the model I'm going to use the Dataset class to adapt all of my data to the model.

In this case I will create 2 instances since each instance will need to be fit with the IDs, which won't be the same across all 2 datasets

In [46]:
dataset=Dataset()
test_dataset=Dataset()

The first thing I need is to fit the Dataset with information about the user and item IDs and their features, so it cant create it's own internal mappings to later generate the matrixes

In [47]:
founders_feature_list=list(df_founders.columns)
cofounders_feature_list=list(df_cofounders.columns)

In [48]:
dataset.fit(users=df_founders.index,items=df_cofounders.index,user_features=founders_feature_list,item_features=cofounders_feature_list)
test_dataset.fit(users=test_df_founders.index,items=test_df_cofounders.index,user_features=founders_feature_list,item_features=cofounders_feature_list)

The LightFM model will take in different sparse matrixes.

The matrixes will be created by the Dataset module from different iterables which we now need to create.

The first matrix we will create is the user-item interaction (founders-cofounders similarity).

In [49]:
founder_cofounder_similarity_iterable=[(founder_id,cofounder_id,founder_cofounder_similarity[founder_id][cofounder_id]) for cofounder_id in df_cofounders.index for founder_id in df_founders.index]
test_founder_cofounder_similarity_iterable=[(test_df_founders.index[founder_id],test_df_cofounders.index[cofounder_id],test_founder_cofounder_similarity[founder_id][cofounder_id]) for cofounder_id in range(len(test_df_cofounders.index)) for founder_id in range(len(test_df_founders.index))]

In [50]:
len(test_founder_cofounder_similarity_iterable)

50000

In [51]:
test_founder_cofounder_similarity[3][0]

0.049415122341757184

In [52]:
test_founder_cofounder_similarity_iterable[3]

(503, 0, 0.049415122341757184)

This method will generate the interactions matrix we will later use

In [53]:
train,weights_matrix=dataset.build_interactions(founder_cofounder_similarity_iterable)
test,test_weights_matrix=test_dataset.build_interactions(test_founder_cofounder_similarity_iterable)

In [54]:
train,test

(<500x100 sparse matrix of type '<class 'numpy.int32'>'
 	with 50000 stored elements in COOrdinate format>,
 <500x100 sparse matrix of type '<class 'numpy.int32'>'
 	with 50000 stored elements in COOrdinate format>)

We're going to finally convert all of the categorical columns of the dataframe into codes, there's no need to normalize these columns or the numerical columns as the dataset instances will normalize the entire dataframe for us

In [55]:
for col in cofounders_categorical_copy:
  df_cofounders[col]=pd.Categorical(df_cofounders[col])
  df_cofounders[col]=df_cofounders[col].cat.codes

for col in founders_categorical_copy:
  df_founders[col]=pd.Categorical(df_founders[col])
  df_founders[col]=df_founders[col].cat.codes

In [56]:
for col in cofounders_categorical_copy:
  test_df_cofounders[col]=pd.Categorical(test_df_cofounders[col])
  test_df_cofounders[col]=test_df_cofounders[col].cat.codes

for col in founders_categorical_copy:
  test_df_founders[col]=pd.Categorical(test_df_founders[col])
  test_df_founders[col]=test_df_founders[col].cat.codes

Now I will create the 2 remaining iterables so I can pass them as parameters to the Dataset methods

In [57]:
cofounders_feature_iterable=[(cofounder_id,{feature_name:df_cofounders[feature_name][cofounder_id]}) for feature_name in df_cofounders.columns for cofounder_id in df_cofounders.index]
founders_feature_iterable=[(founder_id,{feature_name:df_founders[feature_name][founder_id]}) for feature_name in df_founders.columns for founder_id in df_founders.index]

In [58]:
test_cofounders_feature_iterable=[(cofounder_id,{feature_name:test_df_cofounders[feature_name][cofounder_id]}) for feature_name in test_df_cofounders.columns for cofounder_id in test_df_cofounders.index]
test_founders_feature_iterable=[(founder_id,{feature_name:test_df_founders[feature_name][founder_id]}) for feature_name in test_df_founders.columns for founder_id in test_df_founders.index]

Now we will finally create the feature matrixes for founders and cofounders

In [59]:
founders_feature_matrix=dataset.build_user_features(founders_feature_iterable, normalize=True)
cofounders_feature_matrix=dataset.build_item_features(cofounders_feature_iterable, normalize=True)

Test feature matrixes

In [60]:
test_founders_feature_matrix=test_dataset.build_user_features(test_founders_feature_iterable, normalize=True)
test_cofounders_feature_matrix=test_dataset.build_item_features(test_cofounders_feature_iterable, normalize=True)

It is very important to check that event the feature matrixes **share the exact same shape**, as we will receive an error otherwise during the metrics evaluation

In [61]:
founders_feature_matrix,cofounders_feature_matrix

(<500x524 sparse matrix of type '<class 'numpy.float32'>'
 	with 12500 stored elements in Compressed Sparse Row format>,
 <100x123 sparse matrix of type '<class 'numpy.float32'>'
 	with 2400 stored elements in Compressed Sparse Row format>)

In [62]:
test_founders_feature_matrix,test_cofounders_feature_matrix

(<500x524 sparse matrix of type '<class 'numpy.float32'>'
 	with 12500 stored elements in Compressed Sparse Row format>,
 <100x123 sparse matrix of type '<class 'numpy.float32'>'
 	with 2400 stored elements in Compressed Sparse Row format>)

#Model Training

In [63]:
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10,verbose=True)

train_precision = evaluation.precision_at_k(model, train, k=10).mean()
test_precision = evaluation.precision_at_k(model, test, k=10).mean()

#train_auc = evaluation.auc_score(model, train).mean()
#test_auc = evaluation.auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
#print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Epoch: 100%|██████████| 10/10 [29:24<00:00, 176.49s/it]

Precision: train 1.00, test 1.00.


In [64]:
with open('model_lightfm_bpr.pkl', 'wb') as file:
    pickle.dump(model, file)

In [65]:
model = LightFM(learning_rate=0.0005, loss='warp')

model.fit_partial(train, epochs=10,verbose=True)

train_precision = evaluation.precision_at_k(model, train, k=10).mean()
test_precision = evaluation.precision_at_k(model, test, k=10).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Epoch: 100%|██████████| 10/10 [00:00<00:00, 17.15it/s]


Precision: train 1.00, test 1.00.


In [66]:
model.predict([0,0,0,0],[12,5,34,54])

array([-0.00269947, -0.0038101 , -0.00060415,  0.00529658], dtype=float32)

In [67]:
with open('model_lightfm_warp.pkl', 'wb') as file:
    pickle.dump(model, file)

#MODEL EVALUATION

In [68]:
with open('model_lightfm_warp.pkl', 'rb') as file:
    loaded_model_warp = pickle.load(file)

In [69]:
train_precision = evaluation.precision_at_k(loaded_model_warp, train, k=10).mean()
test_precision = evaluation.precision_at_k(loaded_model_warp, test, k=10).mean()

In [70]:
train_precision,test_precision

(1.0, 1.0)

In [71]:
train_reciprocal_rank=evaluation.reciprocal_rank(loaded_model_warp,train)
test_reciprocal_rank=evaluation.reciprocal_rank(loaded_model_warp,test)

In [72]:
train_reciprocal_rank.mean(),test_reciprocal_rank.mean()

(1.0, 1.0)